In [1]:
from pyspark import SparkConf, SparkContext
import os
import re
import random

In [2]:
def k_gram_list(s, shingle_len=3):
    #s = s.replace('-', ' ')
    s = s.lower()
    s = re.sub(r'[^\w\s]','',s)
    tokens = s.split()
    return [tuple(tokens[i:i+shingle_len]) for i in range(len(tokens) - shingle_len+1)]

def get_all_shingles(folder_dir='./athletics'):
    shingles = set()
    sorted_listdir = sorted(os.listdir(folder_dir))
    
    for fl in sorted_listdir:
        fl_path = os.path.join(folder_dir, fl)
        if fl_path[-4:] == '.txt':
            with open(fl_path) as f:
                for line in f:
                    for kgram in k_gram_list(line):
                        shingles.add(kgram) 

    #print(len(shingles)) # 25612
    return  shingles

# get_all_shingles()

In [3]:
def parse_input(f):
    file_name = f[0].split('/')[-1].replace('.txt', '')
    all_kgrams = []
    for line in f[1].split('\n'):
        if line != '':
            line = re.sub(r'\'', '', line)
            all_kgrams.extend(k_gram_list(line))
        
    return file_name, all_kgrams
    

def has_shingle(x):
    s_id, shingle = x[0][0], x[0][1]
    d_id, d_list = x[1][0], x[1][1]
    
    if shingle in d_list:
        return d_id, (s_id, 1)
    else:
        return d_id, (s_id, 0)

In [4]:
conf = SparkConf().setMaster("local").setAppName("pyspark-lsh")
sc = SparkContext.getOrCreate(conf=conf)

folder_dir = './athletics'

""" Shingling """
# (doc_id, [all k_grams tuples])
docs_rdd = sc.wholeTextFiles(folder_dir).map(parse_input).sortByKey()
# docs_rdd.take(5)

shingles = get_all_shingles()

# (shing_id, shingle tuple)
shingles_rdd = sc.parallelize(shingles).zipWithIndex().map(lambda x:(x[1], x[0]))

joined = shingles_rdd.cartesian(docs_rdd) \
        .map(has_shingle)

characteristic_row = joined.groupByKey().sortByKey().cache()

# (doc_id, [shing_ids that (shing_id, doc_id)==1])
filtered = joined.filter(lambda x:x[1][1]==1) \
        .mapValues(lambda x:x[0]).groupByKey()


In [5]:
# randomly generate 100 hash functions to simulate the permutation step
random.seed(42)
a = random.sample(range(1,10000), 100)
b = random.sample(range(1,10000), 100)

N = len(shingles) # 25612
p = 25999 # a prime number > N

num_bands = 50
num_buckets = 10000 # number of buckets to hash to


def min_hash(a, b, shing_ids):
    hashed_rows = [((a*int(r) + b) % p) % N for r in shing_ids]
    return min(hashed_rows)

def get_signature(x):
    doc_id, shing_ids = x
    return [((doc_id, i%num_bands), min_hash(a[i], b[i], shing_ids)) for i in range(100)]

# hash table for 2 rows per band
def band_hasher(x):
    return (((631*x[0]+641*x[1])<<1) % 10099) % num_buckets 

def hash_to_bucket(x):
    doc_id, band_id, sig_list = x[0][0], x[0][1], x[1]
    return ((band_id, band_hasher(sig_list)), [doc_id])

In [6]:
""" Min-hashing """
signatures = filtered.flatMap(get_signature)

""" Locality-sensitive hashing """
# ((d_id, band_id), [sig vals])
bands = signatures.groupByKey().mapValues(list)

bands = bands.map(hash_to_bucket) \
            .reduceByKey(lambda a, b: a + b)\
            .filter(lambda x: len(x[1]) > 1)

#bands.take(20) # ((band_id, bucket_id), [doc_ids])

In [7]:
def candidates_mapper(x):
    similar_set, index = x[0], x[1]
    return map(lambda ele: (ele, index), similar_set)

# use frozenset since our key is a list(mutable)
candidates = bands.map(lambda x: frozenset(sorted(x[1]))).distinct() \
                .zipWithIndex().flatMap(candidates_mapper)
# (doc_id, band_id)
# ('049', 0),('048', 0),('079', 1),('081', 1),('095', 2),('076', 2),...

In [8]:
# add shingles information by doc_id
doc_vectors = candidates.join(characteristic_row) \
                        .map(lambda x: (x[1][0], (x[0], x[1][1])))
# (0, ('049', <pyspark.resultiterable.ResultIterable at 0x10a292c40>))

# same as using groupByKey(), but more efficient
# output: (bucket_id, [(doc_id, characteristic), ...])
bucket_vectors = doc_vectors.mapValues(lambda x:[x]).reduceByKey(lambda a,b:a+b)

# bucket_vectors.sortBy(lambda x: x[0]).take(10)

In [9]:
import itertools
def calculate_jaccard_sim(x):
    bucket_id, cdt_list = x

    idxs = [i for i in range(len(cdt_list))]
    lst = []
    for e1, e2 in itertools.combinations(idxs,2):
        sum_list = [a[1]+b[1] for a, b in zip(cdt_list[e1][1], cdt_list[e2][1])]
        sim = sum_list.count(2) / ( sum_list.count(2)+sum_list.count(1))
        lst.append( ((cdt_list[e1][0], cdt_list[e2][0]), sim)) 

    return lst


bucket_vect_with_sim = bucket_vectors.flatMap(calculate_jaccard_sim)

# get distinct keys, and sort by similarity in decreasing order
ans = bucket_vect_with_sim.reduceByKey(lambda x,y : x).sortBy(lambda x : -x[1]).take(10)
#ans

In [10]:
with open('output.txt', 'w') as of:
    for l in ans:
        print(f'{l[0]}: {l[1]}')
        of.write(f'{l[0]}: {l[1]}\n')

('052', '084'): 1.0
('020', '012'): 1.0
('049', '047'): 0.7576576576576577
('030', '035'): 0.7097097097097097
('049', '088'): 0.5165876777251185
('049', '048'): 0.4839476813317479
('023', '038'): 0.4804177545691906
('040', '014'): 0.40238704177323104
('088', '047'): 0.3917340521114106
('047', '048'): 0.36666666666666664
